In [1]:
import numpy as np
import pandas as pd
import seaborn as sns

import re
import datetime

from pyspark.sql import SparkSession, DataFrame
from pyspark.sql.functions import *
from pyspark.sql.types import DateType

In [2]:
sp = (
    SparkSession.builder.appName("Model")
    .config("spark.sql.session.timeZone", "+11")
    .config("spark.driver.memory", "4g")
    .config("spark.executor.memory", "6g")
    .getOrCreate()
)

22/10/05 21:02:57 WARN Utils: Your hostname, J-L resolves to a loopback address: 127.0.1.1; using 172.18.71.108 instead (on interface eth0)
22/10/05 21:02:57 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address


Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


22/10/05 21:02:58 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [3]:
transactions = sp.read.option("inferSchema", True).parquet("../data/processed/transactions")
merchants = sp.read.option("inferSchema", True).parquet("../data/processed/merchants")
customers = sp.read.option("inferSchema", True).parquet("../data/processed/customers")

In [4]:
transactions.show(1)
merchants.show(1)
customers.show(1)

+--------+-------+------------+------------+--------------+-----------+-----------------+-------+----------+-----+---------+
|order_id|user_id|merchant_abn|dollar_value|order_datetime|Natural_var|Potential_Outlier|holiday|dayofmonth|month|dayofweek|
+--------+-------+------------+------------+--------------+-----------+-----------------+-------+----------+-----+---------+
|       3|      3| 60956456424|      136.68|    2021-08-20|          0|                0|      0|        20|    8|        6|
+--------+-------+------------+------------+--------------+-----------+-----------------+-------+----------+-----+---------+
only showing top 1 row

+------------+-----------------+--------------------+---------------+---------------+----------------+-----------------+
|merchant_abn|             name|                tags|avg_monthly_inc|monthly_entropy|postcode_entropy|          revenue|
+------------+-----------------+--------------------+---------------+---------------+----------------+-------

In [5]:
final = transactions.join(merchants, on="merchant_abn").join(customers, on="user_id")
final.show(2)

+-------+------------+--------+------------+--------------+-----------+-----------------+-------+----------+-----+---------+--------------------+--------------------+---------------+---------------+----------------+------------------+-----+--------+------+--------------------------------------------------+------------------------------+-----------------------------+-------------------------------+---------------------+-----------------------+----------------------+------------------------+--------------+----------------+---------------+----------------------------+---------------------------+------------------------+-----------------------+-------------------------------------+---------------------------+-----------------------------+----------------------------+---------------------------------------+-----------------------------+-------------------------------+------------------------------+-----------------------+-------------+---------------+--------------+-------------------------

In [6]:
final.count()

13614648

### Dropping Columns

In [7]:
final.printSchema()

root
 |-- user_id: long (nullable = true)
 |-- merchant_abn: long (nullable = true)
 |-- order_id: long (nullable = true)
 |-- dollar_value: float (nullable = true)
 |-- order_datetime: date (nullable = true)
 |-- Natural_var: integer (nullable = true)
 |-- Potential_Outlier: integer (nullable = true)
 |-- holiday: long (nullable = true)
 |-- dayofmonth: integer (nullable = true)
 |-- month: integer (nullable = true)
 |-- dayofweek: integer (nullable = true)
 |-- name: string (nullable = true)
 |-- tags: string (nullable = true)
 |-- avg_monthly_inc: float (nullable = true)
 |-- monthly_entropy: float (nullable = true)
 |-- postcode_entropy: float (nullable = true)
 |-- revenue: double (nullable = true)
 |-- state: string (nullable = true)
 |-- postcode: long (nullable = true)
 |-- gender: string (nullable = true)
 |-- Number of individuals lodging an income tax return: long (nullable = true)
 |-- Average taxable income or loss: long (nullable = true)
 |-- Median taxable income or loss

In [13]:
final = final.drop("user_id", "merchant_abn", "order_id", "order_datetime", "name")
final.printSchema()

root
 |-- dollar_value: float (nullable = true)
 |-- Natural_var: integer (nullable = true)
 |-- Potential_Outlier: integer (nullable = true)
 |-- holiday: long (nullable = true)
 |-- dayofmonth: integer (nullable = true)
 |-- month: integer (nullable = true)
 |-- dayofweek: integer (nullable = true)
 |-- tags: string (nullable = true)
 |-- avg_monthly_inc: float (nullable = true)
 |-- monthly_entropy: float (nullable = true)
 |-- postcode_entropy: float (nullable = true)
 |-- revenue: double (nullable = true)
 |-- state: string (nullable = true)
 |-- postcode: long (nullable = true)
 |-- gender: string (nullable = true)
 |-- Number of individuals lodging an income tax return: long (nullable = true)
 |-- Average taxable income or loss: long (nullable = true)
 |-- Median taxable income or loss: long (nullable = true)
 |-- Proportion with salary or wages: long (nullable = true)
 |-- Count salary or wages: long (nullable = true)
 |-- Average salary or wages: long (nullable = true)
 |-- Me

### PROCESSING CUSTOMER FRAUD DATA

In [8]:
c_fraud = sp.read.option("inferSchema", True).parquet("../data/curated/customer_fraud")
c_fraud = c_fraud.withColumn("order_datetime", col("order_datetime").cast(DateType()))
c_fraud.show(2)

+-------+--------------+-----------------+
|user_id|order_datetime|fraud_probability|
+-------+--------------+-----------------+
|   6228|    2021-12-19|         97.62981|
|  21419|    2021-12-10|         99.24738|
+-------+--------------+-----------------+
only showing top 2 rows



In [9]:
transactions.show(2)

+--------+-------+------------+------------+--------------+-----------+-----------------+-------+----------+-----+---------+
|order_id|user_id|merchant_abn|dollar_value|order_datetime|Natural_var|Potential_Outlier|holiday|dayofmonth|month|dayofweek|
+--------+-------+------------+------------+--------------+-----------+-----------------+-------+----------+-----+---------+
|       3|      3| 60956456424|      136.68|    2021-08-20|          0|                0|      0|        20|    8|        6|
|       8|  18482| 70501974849|       68.75|    2021-08-20|          0|                0|      0|        20|    8|        6|
+--------+-------+------------+------------+--------------+-----------+-----------------+-------+----------+-----+---------+
only showing top 2 rows



In [10]:
c_fraud_full = transactions.join(c_fraud, on=["user_id", "order_datetime"])
c_fraud_full.show(2)

+-------+--------------+--------+------------+------------+-----------+-----------------+-------+----------+-----+---------+-----------------+
|user_id|order_datetime|order_id|merchant_abn|dollar_value|Natural_var|Potential_Outlier|holiday|dayofmonth|month|dayofweek|fraud_probability|
+-------+--------------+--------+------------+------------+-----------+-----------------+-------+----------+-----+---------+-----------------+
|    448|    2021-08-20|    1005| 94380689142|     6263.03|          0|                0|      0|        20|    8|        6|        14.681704|
|   3116|    2021-08-20|    6989| 22248828825|     3958.86|          0|                0|      0|        20|    8|        6|         8.809071|
+-------+--------------+--------+------------+------------+-----------+-----------------+-------+----------+-----+---------+-----------------+
only showing top 2 rows



In [11]:
c_fraud_full.count()

80560

In [12]:
X = c_fraud_full.join(merchants, on="merchant_abn").join(customers, on="user_id")
X.show(1)

+-------+------------+--------------+--------+------------+-----------+-----------------+-------+----------+-----+---------+-----------------+-----------+--------------------+---------------+---------------+----------------+----------------+-----+--------+------+--------------------------------------------------+------------------------------+-----------------------------+-------------------------------+---------------------+-----------------------+----------------------+------------------------+--------------+----------------+---------------+----------------------------+---------------------------+------------------------+-----------------------+-------------------------------------+---------------------------+-----------------------------+----------------------------+---------------------------------------+-----------------------------+-------------------------------+------------------------------+-----------------------+-------------+---------------+--------------+------------------

In [13]:
X = X.drop("user_id", "merchant_abn", "order_datetime", "order_id", "name")
X.printSchema()

root
 |-- dollar_value: float (nullable = true)
 |-- Natural_var: integer (nullable = true)
 |-- Potential_Outlier: integer (nullable = true)
 |-- holiday: long (nullable = true)
 |-- dayofmonth: integer (nullable = true)
 |-- month: integer (nullable = true)
 |-- dayofweek: integer (nullable = true)
 |-- fraud_probability: float (nullable = true)
 |-- tags: string (nullable = true)
 |-- avg_monthly_inc: float (nullable = true)
 |-- monthly_entropy: float (nullable = true)
 |-- postcode_entropy: float (nullable = true)
 |-- revenue: double (nullable = true)
 |-- state: string (nullable = true)
 |-- postcode: long (nullable = true)
 |-- gender: string (nullable = true)
 |-- Number of individuals lodging an income tax return: long (nullable = true)
 |-- Average taxable income or loss: long (nullable = true)
 |-- Median taxable income or loss: long (nullable = true)
 |-- Proportion with salary or wages: long (nullable = true)
 |-- Count salary or wages: long (nullable = true)
 |-- Average

Categorical
- holiday (done)
- dayofmonth ?
- dayofweek
- month (done)
- tags
- state
- gender
- postcode

In [14]:
from pyspark.ml.feature import StringIndexer, OneHotEncoder

from pyspark.ml import Pipeline
from pyspark.sql.types import FloatType
from pyspark.mllib.evaluation import MulticlassMetrics
from pyspark.ml.regression import GeneralizedLinearRegression, GBTRegressor
from pyspark.ml.classification import MultilayerPerceptronClassifier
from pyspark.ml.feature import StringIndexer, OneHotEncoder, VectorAssembler, StandardScaler
from pyspark.ml.evaluation import MulticlassClassificationEvaluator, RegressionEvaluator

def category_processing(data: DataFrame, outcome: str):
    categories = [
        "dayofmonth",
        "dayofweek",
        "month",
        "tags",
        "state",
        "gender",
        "postcode"
    ]

    # Pipeline
    indexers = [StringIndexer(inputCol=c, outputCol=c+"_index") for c in categories]
    encoders = [OneHotEncoder(inputCol=c+"_index", outputCol=c+"_encoded") for c in categories]
    transformed = Pipeline(stages=indexers + encoders).fit(data).transform(data)

    for c in categories:
        transformed = transformed.drop(c).drop(c+"_index")
    return transformed

In [16]:
category_processed = category_processing(X, "outcome")
category_processed.show(1)

+------------+-----------+-----------------+-------+-----------------+---------------+---------------+----------------+----------------+--------------------------------------------------+------------------------------+-----------------------------+-------------------------------+---------------------+-----------------------+----------------------+------------------------+--------------+----------------+---------------+----------------------------+---------------------------+------------------------+-----------------------+-------------------------------------+---------------------------+-----------------------------+----------------------------+---------------------------------------+-----------------------------+-------------------------------+------------------------------+-----------------------+-------------+---------------+--------------+----------------------------------+------------------------------------+-----------------------------------+------------------+-----------------+

## TRAIN TEST SPLIT

In [17]:
y = X.select("fraud_probability")
y.count()

71813

In [19]:
y.describe().show()

+-------+------------------+
|summary| fraud_probability|
+-------+------------------+
|  count|             71813|
|   mean|14.717076242080452|
| stddev| 9.404555316490011|
|    min|          8.287144|
|    max|          97.62981|
+-------+------------------+



In [26]:
from pyspark.ml.feature import Bucketizer

buckets = Bucketizer(splits=[0, 10, 20, 30, 40, 50, 60, 70, 80, 90, 100], inputCol="fraud_probability", outputCol="fraud_buckets")
X_bucks = buckets.transform(X)

X_bucks.show(5)

+------------+-----------+-----------------+-------+----------+-----+---------+-----------------+--------------------+---------------+---------------+----------------+------------------+-----+--------+-----------+--------------------------------------------------+------------------------------+-----------------------------+-------------------------------+---------------------+-----------------------+----------------------+------------------------+--------------+----------------+---------------+----------------------------+---------------------------+------------------------+-----------------------+-------------------------------------+---------------------------+-----------------------------+----------------------------+---------------------------------------+-----------------------------+-------------------------------+------------------------------+-----------------------+-------------+---------------+--------------+----------------------------------+---------------------------------

In [27]:
X_bucks.groupBy("fraud_buckets").count().orderBy("fraud_buckets").show()

+-------------+-----+
|fraud_buckets|count|
+-------------+-----+
|          0.0|22923|
|          1.0|38611|
|          2.0| 6113|
|          3.0| 1934|
|          4.0|  990|
|          5.0|  576|
|          6.0|  360|
|          7.0|  193|
|          8.0|  102|
|          9.0|   11|
+-------------+-----+



In [40]:
from functools import reduce

X_bucks_9 = X_bucks.filter(X_bucks["fraud_buckets"] == 9.0)
X_bucks_8 = X_bucks.filter(X_bucks["fraud_buckets"] == 8.0)
X_bucks_7 = X_bucks.filter(X_bucks["fraud_buckets"] == 7.0)
X_bucks_6 = X_bucks.filter(X_bucks["fraud_buckets"] == 6.0)
X_bucks_5 = X_bucks.filter(X_bucks["fraud_buckets"] == 5.0)
X_bucks_4 = X_bucks.filter(X_bucks["fraud_buckets"] == 4.0)
X_bucks_3 = X_bucks.filter(X_bucks["fraud_buckets"] == 3.0)

X_bucks_9_over = X_bucks_9.sample(withReplacement=True, fraction=350.0, seed=69)
X_bucks_8_over = X_bucks_8.sample(withReplacement=True, fraction=35.0, seed=69)
X_bucks_7_over = X_bucks_7.sample(withReplacement=True, fraction=20.0, seed=69)
X_bucks_6_over = X_bucks_6.sample(withReplacement=True, fraction=15.0, seed=69)
X_bucks_5_over = X_bucks_5.sample(withReplacement=True, fraction=7.0, seed=69)
X_bucks_4_over = X_bucks_4.sample(withReplacement=True, fraction=4.0, seed=69)
X_bucks_3_over = X_bucks_3.sample(withReplacement=True, fraction=2.0, seed=69)

X_adjusted = reduce(DataFrame.unionAll, [X_bucks_9_over, X_bucks_8_over, X_bucks_7_over, X_bucks_6_over, X_bucks_5_over, X_bucks_4_over, X_bucks_3_over])
X_adjusted = reduce(DataFrame.unionAll, [X_adjusted, X_bucks.filter(X_bucks["fraud_buckets"] == 2.0), X_bucks.filter(X_bucks["fraud_buckets"] == 1.0), X_bucks.filter(X_bucks["fraud_buckets"] == 0.0)])
X_adjusted.count()

96272

In [41]:
X_adjusted.groupBy("fraud_buckets").count().orderBy("fraud_buckets").show()

+-------------+-----+
|fraud_buckets|count|
+-------------+-----+
|          0.0|22923|
|          1.0|38611|
|          2.0| 6113|
|          3.0| 3910|
|          4.0| 3931|
|          5.0| 4025|
|          6.0| 5450|
|          7.0| 3876|
|          8.0| 3583|
|          9.0| 3850|
+-------------+-----+



In [44]:
train, val, test = X_adjusted.randomSplit([0.7, 0.2, 0.1], seed=69)

print(train.count())
print(val.count())
test.count()

67492


19067


9713

In [ ]:
def process_numerical(data: DataFrame):
    """
    Function to scale and process numerical columns
    """
    # Scaler
    columns = ["Temperature", "Dew Point", "Humidity", "Wind Speed", "Wind Gust", "Pressure", "Precipitation"]

    va = VectorAssembler(inputCols=columns, outputCol="to_scale")
    sc = StandardScaler(inputCol="to_scale", outputCol="scaled")

    va_data = va.transform(data)
    data = sc.fit(va_data).transform(va_data)
    # Drop other columns
    for c in columns:
        data = data.drop(c)
    return data.drop("to_scale")